In [1]:
#Importy do całego kodu
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import os
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar
from IPython.display import clear_output
from ignite.handlers import FastaiLRFinder
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
import plotly.graph_objects as go
from plotly.subplots import make_subplots

/SI/si/lib/python3.11/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


Poniżej znajduje się kod na wczytanie plików dźwiękowych, przetworzenie ich i zapisanie przetworzonych danych do dalszego użycia.
Kod bazuje na kodzie z przykładu, ale przy pętli, która przeszukuje foldery dodałem algorytm sprawdzający, czy to rzeczywiście folder, bo w tej ścieżce znajdują się również pliki. Tak samo potem sprawdzam, czy pliki to .wav, bo pojawiały się też inne pliki. Ustawiłem dtype na float32, bo domyślnie był to float64 i logfbank_feats zajmował około 20GB, przejście na float32 ścieło rozmiar o około połowę. Inną opcją było zmniejszenie ilości plików z każdego folderu do na przykład 200 zamiast 300, ale nie chciałem mieć mniej obiektów do szkolenia. 

In [ ]:
paths = []
labels = []
labels_categorical = []
root = 'SpeechDataset/'

# Iteracja przez katalogi w folderze głównym
for ind, subdir in enumerate(os.listdir(root)):
    subdir_path = os.path.join(root, subdir)
    if not os.path.isdir(subdir_path):
        continue
    for file in os.listdir(os.path.join(root, subdir))[:300]:
        filepath = os.path.join(root, subdir, file)
        if not filepath.endswith('.wav'):
            continue
        paths.append(filepath)
        labels.append(ind)
        labels_categorical.append(subdir)

print(f"Found {len(paths)} WAV files")

# Obliczanie cech logfbank dla każdego sygnału
logfbank_feats = []
for signal_path in paths:
    fs, sig = wav.read(signal_path)
    fbank_feat = logfbank(sig, samplerate=fs)
    logfbank_feats.append(fbank_feat)

# Ustalenie maksymalnej długości sygnału i wyrównanie długości
lengths = [i.shape[0] for i in logfbank_feats]
max_len = np.max(lengths)
print(f"Max length: {max_len}")

# Tworzenie macierzy o ustalonej maksymalnej długości
padded_feats = np.zeros((len(lengths), max_len, logfbank_feats[0].shape[1]), dtype=np.float32)
for i, feats in enumerate(logfbank_feats):
    padded_feats[i, :, :] = np.pad(feats, 
                                    ((int(np.floor((max_len - feats.shape[0]) / 2)),
                                      int(np.ceil((max_len - feats.shape[0]) / 2))),
                                     (0, 0)))

print(f"padded_feats shape: {padded_feats.shape}, dtype: {padded_feats.dtype}")

# Zapisanie wyników do plików
np.save('logfbank_feats.npy', padded_feats)
np.save('labels.npy', labels)
np.save('labels_categorical.npy', labels_categorical)


Wczytanie zapisanych plików oraz podział zbiorów.
Wczytuję pliki z przedrostkiem "t_", ponieważ Pani Tośka uzyskała z jakiegoś powodu pliki idealne, które zajmują tylko około 116MB i przetważają się dużo szybciej (moje około 8 minut przy 150 epokach, a tutaj to kwestia sekund). Mają też o wiele mniej wierszy, ale to właśnie taki wymiar, do jakiego dostosowany jest kod z przykładu (z jakiegoś powodu ja, Łukasz, Alina i Jan otrzymywaliśmy 100 razy więcej wierszy w danych, stąd 20GB). Dodam, że Pani Tośka korzystała z identycznego kodu na wygenerowanie tych plików jak ja. Jedyna różnica polega na tym, że zrobiła to na colabie, ale potem wykonała kod w vscode i uzyskała jeszcze 2x mniejszy plik od tych idealnych.
Różnica w wynikach nie była aż tak wielokrotna, przejście na pliki Pani Tośku pozwoliło mi uzyskać o 10% większe accuracy. 
Aktualizacja: Dopiero podczas opisywania dalszej części kodu zauważyłem, że pliki te mają tylko 21 klas, podczas gdy moje pliki miały zawsze 42 klasy, a pliki Pana Łukasza 36 klas (36 to wartość, która pojawiała się również w przykładach)

In [2]:
# Załadowanie danych
feats = np.load('t_logfbank_feats.npy')
labels = np.load('t_labels.npy')

# Przekształcenie danych
feats = feats.reshape(feats.shape[0], -1)
feats = feats.astype(np.float32)

# Podział na zbiory uczący, walidacyjny i testowy
X_train, X_val_test, y_train, y_val_test = train_test_split(    
    feats,
    labels,
    random_state=1,
    stratify=labels,
    train_size=0.8
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val_test,
    y_val_test,
    random_state=1,
    stratify=y_val_test,
    train_size=0.5
)

Przetworzenie danych z formatu numpy na tensory.

In [3]:
trainset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
valset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
testset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(trainset, batch_size=256)
val_loader = DataLoader(valset, batch_size=256)
test_loader = DataLoader(testset, batch_size=256)

Definicja modelu sieci z dropout, wraz z definicjami kryterium oceny, optymalizatora i trenera.
Przekazuję input_size zmienną ze względu na to, że zmieniałem parametry w pierwszej komórce na różne przed przejściem na pliki Pani Tośki, zmieniał się też wtedy czasem input_size
Opisuję to wszystko w niedzielę, podczas gdy kod tworzyłem tydzień wcześniej, więc nie wszystkie zmiany pamietam, ale wydaje mi się, że ustawiłem inne wartości dropout, dodałem dwie warstwy i zmieniłem optymalizator na Adam oraz zmniejszyłem lr z 3e-3 na 3e-4 w odniesieniu do kodu z przykładu

In [4]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # Pierwsza warstwa gęsta
        self.fc2 = nn.Linear(256, 128)   # Druga warstwa gęsta
        self.fc3 = nn.Linear(128, 64)   # Trzecia warstwa gęsta
        self.fc4 = nn.Linear(64, 21)    # Czwarta warstwa gęsta
        self.dropout1 = nn.Dropout(p = 0.5)  # Pierwsza warstwa dropout (50% prawdopodobieństwo wyłączenia neuronów)
        self.dropout2 = nn.Dropout(p = 0.35)  # Druga warstwa dropout (35% prawdopodobieństwo wyłączenia neuronów)
        self.dropout3 = nn.Dropout(p = 0.2) # Trzecia warstwa dropout (20% prawdopodobieństwo wyłączenia neuronów)

    def forward(self, x):
        x = self.fc1(x)  # Przechodzimy przez pierwszą warstwę
        x = self.dropout1(x)  # Zastosowanie dropout po pierwszej warstwie
        x = self.fc2(x)  # Przechodzimy przez drugą warstwę
        x = self.dropout2(x)  # Zastosowanie dropout po drugiej warstwie
        x = self.fc3(x)  # Przechodzimy przez trzecią warstwę
        x = self.dropout3(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim = 1)  # Zastosowanie logarytmu softmax na wyjściu

device = "cuda" if torch.cuda.is_available() else "cpu"  # Sprawdza, czy dostępna jest karta graficzna (GPU), jeśli nie, używa CPU
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.NLLLoss()  # Definiuje funkcję strat (strata logarytmiczna z prawdopodobieństw)
# device='cpu'

i_s = X_train.shape[1]

model = Net(i_s)  # Inicjalizuje model sieci neuronowej
model.to(device)  # Przenosi model na odpowiednie urządzenie (GPU lub CPU)
optimizer = optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))  # Definiuje optymalizator SGD z określoną szybkością uczenia i momentem

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

# Tworzymy obiekt trenera
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

# Tworzymy obiekt ewaluatora
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.NLLLoss())}, 
                                         device=device)

# Dodajemy pasek postępu do trenera
ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

CUDA: True


/SI/si/lib/python3.11/site-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Trenowanie i ocena modelu
W odniesieniu do kodu z przykładu dodałem clear_output dla wygody w przeglądaniu notatnika

In [5]:
@trainer.on(Events.EPOCH_COMPLETED)
# Walidacja ma być przeprowadzona po zakończeniu każdej epoki
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

# Trening modelu przez 150 epok
trainer.run(train_loader, max_epochs=100)

# Ewaluacja modelu na zbiorze testowym
evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.4186851211072664, 'loss': 1.903396342452422}


Przejście na AdamW, zmniejszenie szans na deaktywację w dropout, zastosowanie funkcji aktywacji relu, zmiana straty na crossentropy (wybór ze względu na obecność wielu klas na wyjściu).
Zmiany te pozwoliły na zwiększenie accuracy o około 20%.
Dodam, że x pierwszych modeli było początkowo wykonywanych na moich plikach zamiast plikach t_, ale ponieważ kolejne były wykonywane na plikach t_, to wykonałem ponownie te kody na plikach t_ dla lepszego odniesienia. Wyniki na moich plikach były około 10-15% gorsze.

In [11]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 21)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6141868512110726, 'loss': 1.3253483227792495}


Szczerze pryznaję, że nie wiem, co zmieniłem w poniższym kodzie w porównaniu do powyższego, ale za każdym razem mam 1-3% większy wynik accuracy z poniższego kodu

In [14]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 21)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6470588235294118, 'loss': 1.234823787913603}


Poniżej nastąpiła próba dodania informacji zwrotnych w warstwach, aczkolwiek nie byłem jeszcze świadom, że robię to niepoprawnie, bo w jednym wykonaniu dwa razy przechodzę przez 2 i 3 warstwę, zamiast informować warstwę w przyszłych wykonaniach o wynikach z poprzednich. Wyniki porównywalne do powyższego kodu

In [16]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 21)
        self.fc_feedback1 = nn.Linear(64, 256)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x):
        x1 = F.relu(self.fc1(x))
        x1 = self.dropout1(x1)
        x2 = F.relu(self.fc2(x1))
        x2 = self.dropout2(x2)
        x3 = F.relu(self.fc3(x2))
        x3 = self.dropout3(x3)
        x2_feedback = F.relu(self.fc_feedback1(x3)) 
        x2 = F.relu(self.fc2(x2_feedback+x1))
        x2 = self.dropout2(x2)
        x3 = F.relu(self.fc3(x2))
        x3 = self.dropout3(x3)
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6470588235294118, 'loss': 1.292795253872459}


Ponownie myślałem, że dobrze zaimplementowałem, ale przecież przy każdym wywołaniu forward zmienne będą resetowane do zera, więc nadal jest źle, aczkolwiek wynik z jakiegoś powodu jest o 3% większy. Zauważyłem ten błąd myśląc o kodzie zaraz po ostatnich zajęciach, ale potem o tym zapomniałem i przypomniałem sobie o tym dopiero teraz.

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 21)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.relu(self.fc1(x))
        x2 = F.relu(self.fc2(x1+x2_temp))
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.relu(self.fc3(x2+x3_temp))
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6730103806228374, 'loss': 1.1217377870553094}


Poniższy kod wydaje mi się, że powinien być odpowiedni, ale z nieznanego mi powodu otrzymuję błędy. Github Copilot nie był w stanie mi pomóc z tym.

In [33]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 21)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)
        self.feedback1 = nn.Linear(128, 256)
        self.feedback2 = nn.Linear(64, 128)
        self.x2_temp = 0
        self.x3_temp = 0

    def forward(self, x):
        x1 = F.relu(self.fc1(x))
        x2 = F.relu(self.fc2(x1+self.x2_temp))
        x2 = self.dropout2(x2)
        self.x2_temp = self.feedback1(x2).detach()
        x3 = F.relu(self.fc3(x2+self.x3_temp))
        x3 = self.dropout3(x3)
        self.x3_temp = self.feedback2(x3).detach()
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

CUDA: True


[1/19]   5%|5          [00:00<?]

Current run is terminating due to exception: The size of tensor a (16) must match the size of tensor b (256) at non-singleton dimension 0
Engine run is terminating due to exception: The size of tensor a (16) must match the size of tensor b (256) at non-singleton dimension 0


RuntimeError: The size of tensor a (16) must match the size of tensor b (256) at non-singleton dimension 0

Przejście do porównywania różnych funkcji aktywacji.
Najpierw przedstawienie ich różnic na wykresach plotly:

In [39]:
# Zakres wartości wejściowych
x = np.linspace(-5.0, 10.0, 10)

# Funkcje aktywacji
relu = F.relu(torch.tensor(x)).numpy()
sigmoid = torch.sigmoid(torch.tensor(x)).numpy()
tanh = torch.tanh(torch.tensor(x)).numpy()
leaky_relu = F.leaky_relu(torch.tensor(x), negative_slope=0.01).numpy()
softmax = F.softmax(torch.tensor(x), dim=0).numpy()
log_softmax = F.log_softmax(torch.tensor(x), dim=0).numpy()

fig = make_subplots(rows=2, cols=3, subplot_titles=("ReLU", "Sigmoid", "Tanh", "Leaky ReLU", "Softmax", "Log Softmax"))

# Dodawanie wykresów
fig.add_trace(go.Scatter(x=x, y=relu, mode='lines', name='ReLU'), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=sigmoid, mode='lines', name='Sigmoid'), row=1, col=2)
fig.add_trace(go.Scatter(x=x, y=tanh, mode='lines', name='Tanh'), row=1, col=3)
fig.add_trace(go.Scatter(x=x, y=leaky_relu, mode='lines', name='Leaky ReLU'), row=2, col=1)
fig.add_trace(go.Scatter(x=x, y=softmax, mode='lines', name='Softmax'), row=2, col=2)
fig.add_trace(go.Scatter(x=x, y=log_softmax, mode='lines', name='Log Softmax'), row=2, col=3)

# Aktualizacja układu
fig.update_layout(height=600, width=900, title_text="Funkcje Aktywacji", showlegend=False,
                  yaxis1=dict(range=[-0.1,10]),
                  yaxis2=dict(range=[0,1.2]),
                  yaxis3=dict(range=[-1.5,1.5]),
                    yaxis4=dict(range=[-1,1]),
                    yaxis5=dict(range=[-0.1,2]),
                    xaxis5=dict(range=[-5,10]),
                    yaxis6=dict(range=[-15,1]))

fig.show()

Wszystkie miały gorsze wyniki od relu i leaky_relu.
ReLU zwraca wejścia, jeśli jest dodatnie, a w przeciwnym wypadku zero

Softmax przekształca wektor wartości w rozkład prawdopodobieństwa sumujący się do 1

Softmax 5%

In [20]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.softmax(self.fc1(x))
        x2 = F.softmax(self.fc2(x1+x2_temp))
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.softmax(self.fc3(x2+x3_temp))
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.05190311418685121, 'loss': 3.367358778587262}


Sigmoid przekształca wejście w wartość z przedziału (0,1)

Sigmoid 24%

In [8]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = torch.sigmoid(self.fc1(x))
        x2 = torch.sigmoid(self.fc2(x1+x2_temp))
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = torch.sigmoid(self.fc3(x2+x3_temp))
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.2491349480968858, 'loss': 2.3500845622026385}


Tanh przekształca wejście w wartość z przedziału (-1, 1), centralizując dane wokół zera

Tanh 44%

In [9]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = torch.tanh(self.fc1(x))
        x2 = torch.tanh(self.fc2(x1+x2_temp))
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = torch.tanh(self.fc3(x2+x3_temp))
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.4411764705882353, 'loss': 1.7960610571204585}


Leaky ReLU zwraca wejście, jeśli jest dodatnie, w przeciwnym razie zwraca małą ujemną wartość proporcjonalną do wejścia.

Leaky_relu, jak wspomniałem daje rozsądne wyniki, 66%

In [43]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.8, 0.888))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6660899653979239, 'loss': 1.1511010826665224}


CrossEntropyLoss łączy LogSoftmax i NLLLoss, karząc większe różnice między przewidywanymi a rzeczywistymi klasami

NLLoss oblicza negatywną logarytmiczną wartość prawdopodobieństwa dla przewidywanych klas

Powrót do straty NLLLoss, o dziwo lepsze wyniki od CrossEntropy, wydawało mi się, że na zajeciach były gorsze, ale też tutaj leaky_relu, a powyżej gdzie zmieniałem na crossentropy i miałem z tego korzyść miałem relu. Miałem tutaj wyniki takie jak 68% i 69%, ale wykonałem kolejny raz i zmieniło się na 66% i nie byłem w stanie ich odtworzyć.

In [72]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return F.log_softmax(x, dim=1)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.NLLLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.NLLLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6539792387543253, 'loss': 1.1723927428565635}


NLLLoss2d odmiana NLLLoss dla danych 2d

NLLLoss2d, 64%, z jakiegoś powodu nazwa w kodzie jest przekreślona

In [46]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return F.log_softmax(x, dim=1)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.NLLLoss2d()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.NLLLoss2d())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6401384083044983, 'loss': 1.1655193183661332}


MultiMarginLoss oblicza stratę na podstawie marginesów między klasami

MultiMarginLoss, 63%

In [76]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return F.log_softmax(x, dim=1)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.MultiMarginLoss()

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.MultiMarginLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6366782006920415, 'loss': 0.08644909842204057}


FocalLoss modyfikuje CrossEntropyLoss, aby skupić się na trudniejszych do sklasyfikowania przykładach, zmniejszając wpływ łatwych przykładów

FocalLoss, 65%

In [27]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return F.log_softmax(x, dim=1)
    
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(FocalLoss(alpha=1, gamma=2, reduction='mean'))}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6539792387543253, 'loss': 0.8594459612889274}


LabelSmoothingLoss wprowadza wygładzanie klas, aby zmniejszyć pewność modelu co do przewidywań

LabelSmoothingLoss, 67%

In [32]:
from ignite.engine import Engine
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)
        self.dropout3 = nn.Dropout(p=0.1)

    def forward(self, x, x2_temp=0, x3_temp=0):
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x2 = F.leaky_relu(self.fc2(x1+x2_temp), negative_slope=0.01)
        x2 = self.dropout2(x2)
        x2_temp = x2
        x3 = F.leaky_relu(self.fc3(x2+x3_temp), negative_slope=0.01)
        x3 = self.dropout3(x3)
        x3_temp = x3
        x = self.fc4(x3)
        return F.log_softmax(x, dim=1)

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = LabelSmoothingLoss(classes=42, smoothing=0.1)

i_s = X_train.shape[1]

model = Net(i_s)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(LabelSmoothingLoss(classes=42, smoothing=0.1))}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6764705882352942, 'loss': 1.6449754972358888}


Wyszukiwanie najlepszego lr_rate i trening z tym lr, w odniesieniu do przykładu musiałem dodać start_lr, end_lr i num_iter, żeby kod wykonywał się bez błędów. Dzięki finderowi udało mi się uzyskać 70%, rekordowo 70,8%, ale nie udało mi się teraz powtórzyć, a nie mam czasu, żeby wykonywać wiele razy i czekać na ten wynik.

In [63]:
# Inicjalizacja obiektu FastaiLRFinder
lr_finder = FastaiLRFinder()

# Przygotowanie słownika z modelem i optymalizatorem
to_save = {'model': model, 'optimizer': optimizer}

# Dołączenie lr_finder do trenera i ustawienie wartości diverge_th na 1.1
with lr_finder.attach(trainer, to_save, diverge_th=1.1, start_lr=1e-6, end_lr=100.0, num_iter=600) as trainer_with_lr_finder:
    # Domyślnie start_lr jest taki, jak określono w obiekcie optimizer, 
    # a end_lr = 10
    trainer_with_lr_finder.run(train_loader)

# Pobranie wyników po zakończeniu poszukiwań najlepszego LR
results = lr_finder.get_results()

# Wykres przedstawiający wyniki
lr_finder.plot()

# Wyświetlenie sugerowanego współczynnika uczenia
print("Suggested LR:", lr_finder.lr_suggestion())

# Utworzenie obiektów trainer i evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model,
    metrics={"acc": Accuracy(), "loss": Loss(nn.CrossEntropyLoss())},
    device=device
)

# Dodanie paska postępu do trenera
ProgressBar(persist=True).attach(
    trainer, output_transform=lambda x: {"batch loss": x}
)

# Funkcja walidacji po zakończeniu epoki
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)  # Przeprowadzenie walidacji
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch: {trainer.state.epoch} "
          f"Avg accuracy: {metrics['acc']:.2f} "
          f"Avg loss: {metrics['loss']:.2f}")
    
    # Zastosowanie sugerowanego współczynnika uczenia
    lr_finder.apply_suggested_lr(optimizer)
    print(f'Training with suggested lr: {optimizer.param_groups[0]["lr"]}')

# Rozpoczęcie treningu
trainer.run(train_loader, max_epochs=50)

# Ewaluacja na zbiorze testowym
evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.7006920415224913, 'loss': 1.0664540855117324}


Udało mi się również uzyskać przyzwoite wyniki na transformerze

In [ ]:
class TransformerNet(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, num_decoder_layers):
        super(TransformerNet, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, batch_first=True)
        self.fc1 = nn.Linear(d_model, 128)
        self.fc2 = nn.Linear(128, 42)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = self.transformer(x, x)
        x = x.mean(dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'CUDA: {torch.cuda.is_available()}')
criterion = nn.CrossEntropyLoss()

input_size = X_train.shape[1]
d_model = 128
nhead = 8
num_encoder_layers = 2
num_decoder_layers = 2
model = TransformerNet(input_size, d_model, nhead, num_encoder_layers, num_decoder_layers)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, 
                                         metrics={"acc": Accuracy(), 
                                                  "loss": Loss(nn.CrossEntropyLoss())}, 
                                         device=device)

ProgressBar(persist=True).attach(trainer, 
                                  output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {} Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
        trainer.state.epoch, metrics['acc'], metrics['loss']))

trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
clear_output()
print(evaluator.state.metrics)

{'acc': 0.6868512110726643, 'loss': 1.3529279768260705}
